### Import and check

In [36]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import PIL 
from PIL import Image

import torch
import torchvision.ops.boxes as bops

import time

import cv2
from patched_yolo_infer import (visualize_results_usual_yolo_inference)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
!nvidia-smi
torch.cuda.is_available()

Sun Oct 13 00:39:38 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 556.13                 Driver Version: 556.13         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   70C    P0             42W /   50W |    3879MiB /   4096MiB |     93%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

True

In [6]:
#настроим random_seed всего
seed_num = 13
torch.manual_seed(seed_num)
torch.cuda.manual_seed(seed_num)
torch.cuda.manual_seed_all(seed_num)
torch.use_deterministic_algorithms(True) # do not work in collab
np.random.seed(seed_num)

import torch.backends.cudnn
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True # do not work in collab

### Paths

In [7]:
abs_path = os.getcwd()
project_path = r'F:\DS\Хахатон_10.2024'
project_path

'F:\\DS\\Хахатон_10.2024'

In [10]:
# test path

val_list = os.listdir(os.path.join(project_path, 'large_dataset', 'val', 'images'))

plane_img_path = os.path.join(project_path, 'Dataset', 'Dataset_photo', '0-plane', 'images')
heli_img_path = os.path.join(project_path, 'Dataset', 'Dataset_photo', '1-helicopter', 'images')
     
plane_img_listdir = os.listdir(plane_img_path)
heli_img_listdir = os.listdir(heli_img_path)

plane_label_path = os.path.join(project_path, 'Dataset', 'Dataset_photo', '0-plane', 'labels')
heli_label_path = os.path.join(project_path, 'Dataset', 'Dataset_photo', '1-helicopter', 'labels')

plane_label_listdir = os.listdir(plane_label_path)
heli_label_listdir = os.listdir(heli_label_path)

In [ ]:
# test dataset
import pyzipper

zip_file_path = os.path.join(project_path, 'train_dataset_minpromtorg_train_dataset.zip') #!!!!!!!
password = b'cifrovoy_proryv_2024_mpt_bpla' #!!!!!!!!

def decrypt(zip_file_path, password):
    with pyzipper.AESZipFile(zip_file_path, 'r', compression=pyzipper.ZIP_LZMA, encryption=pyzipper.WZ_AES) \
            as extracted_zip:
        try:
            extracted_zip.extractall(pwd = password)
        except RuntimeError as ex:
            print(ex)


decrypt(zip_file_path, password)

### Models

In [91]:
# detection model
from ultralytics import YOLO
#detector = YOLO(os.path.join(os.curdir, 'runs', 'detect', 'train16', 'weights', 'best.pt')) # middle model
detector = YOLO(os.path.join(os.curdir, 'runs', 'detect', 'train18', 'weights', 'best.pt')) # large model

In [ ]:
# classifier
classifier = .....

### Inference

In [72]:
# check time for single image

# start time
start_time = time.time()

res = detector.predict(source = os.path.join(heli_img_path, '27042021_¡¿¿» óÑαΓ«½ÑΓ@ds-0507.jpg'), 
                       show = False,                                                  
                       imgsz = 640,   
                       show_boxes = False,
                       show_labels = False,                                           
                       save = False,                                                  
                       name = "YOLO10_res",                                                 
                       conf = 0.3,   
                       max_det = 2)

# final time
end_time = time.time()

# start time minus final time
elapsed_time = end_time - start_time
print('Elapsed time: ', elapsed_time)


image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\1-helicopter\images\27042021_ @ds-0507.jpg: 512x640 1 class_0, 47.5ms
Speed: 0.0ms preprocess, 47.5ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 640)
Elapsed time:  0.08983683586120605


##### Detection

In [105]:
# get bboxes from the model

def yolo_fast_inference(path, image_name, conf, max_det):
    res = detector.predict(source = os.path.join(path, image_name), 
                        show = False,                                                  
                        imgsz = 640,                                                 
                        show_boxes = False,
                        show_labels = False,                                            
                        save = False,                                                  
                        #name = "YOLO10_res",                                                 
                        conf = conf,   
                        max_det = max_det)
    
    result = res[0]
    if len(res[0].boxes) > 0: #### add several bboxes!!!!!??
        box = result.boxes[0]
        # bboxes
        coords_xyxy = box.xyxy[0].tolist() # bbox_xyxy (upper-left, upper-left, bottom-right, bottom-right)
        coords_xywhn = box.xywhn[0].tolist() # bbox_xywhn (upper-left, upper-left, width, height: normalized)
        # tensor for metric
        pred_tensor = torch.tensor([coords_xyxy], dtype=torch.float) # for metric
    else:
        coords_xyxy = [0, 0, 0, 0] # bbox_xyxy
        coords_xywhn = [0, 0, 0, 0] # bbox_xywhn
        pred_tensor = torch.tensor([[0, 0, 0, 0]], dtype=torch.float) # for metric
        
    return pred_tensor, coords_xyxy, coords_xywhn

In [106]:
# get bboxes from label

def label_recalc(label_list, label_path, label_name):
    if label_name in label_list:
        with open(os.path.join(label_path, label_name), 'r') as file:
            res_test = file.read()
            res_test_list = [float(x) for x in res_test.split(' ')]
            
            # coordinates xyxy
            x_min = res_test_list[1]*img_w - 0.5*(res_test_list[3]*img_w) # center_x to upper_left_x
            y_min = res_test_list[2]*img_h - 0.5*(res_test_list[4]*img_h) # center_x to upper_left_y
            x_max = x_min + res_test_list[3]*img_w # width to bottom-right
            y_max = y_min + res_test_list[4]*img_h # height to bottom-right
            
            # coordinates xywhn
            x_min_norm = res_test_list[1] - 0.5*(res_test_list[3]) # center_x to upper_left_x
            y_min_norm = res_test_list[2] - 0.5*(res_test_list[4]) # center_x to upper_left_y
            width_norm = res_test_list[3] # width 
            height_norm = res_test_list[4] # height 

            test_tensor = torch.tensor([[x_min, y_min, x_max, y_max]], dtype=torch.float) # for metric
            coords_xyxy_label = [x_min, y_min, x_max, y_max] # bbox_xyxy (upper-left, upper-left, bottom-right, bottom-right)
            coords_xywhn_label = [x_min_norm, y_min_norm, width_norm, height_norm] # bbox_xywhn (upper-left, upper-left, width, height: normalized)
    else:
        coords_xyxy_label = [0, 0, 0, 0] # bbox_xyxy
        coords_xywhn_label = [0, 0, 0, 0] # bbox_xywhn
        test_tensor = torch.tensor([[0, 0, 0, 0]], dtype=torch.float) # for metric
    
    return test_tensor, coords_xyxy_label, coords_xywhn_label

In [94]:
# check total time on large_val

# start time
start_time = time.time()
print('Starting...')

name_list = []
coord_list_xywhn_inference = []
coord_list_xyxy_inference = []
coord_list_xyxy_label = []
iou_list = []

conf = 0.3
max_det = 1

for i in range(len(val_list)):
    name_base = val_list[i][:-4:]
    image_name = val_list[i]
    label_name = name_base + '.txt'
    
    # save name for res_table
    name_list.append(val_list[i])
    
    if val_list[i] in plane_img_listdir:
        img_path = plane_img_path
        label_path = plane_label_path
        label_list = plane_label_listdir        
    else:
        img_path = heli_img_path
        label_path = heli_label_path
        label_list = heli_label_listdir        

    # get inference coords
    pred_tensor, coords_xyxy, coords_xywhn = yolo_fast_inference(img_path, image_name, conf, max_det) # or xyxy
    
    # save coords for res_table
    coord_list_xywhn_inference.append(coords_xywhn)
    coord_list_xyxy_inference.append(coords_xyxy)
    
    # get label
    test_tensor, coords_xyxy_label, coords_xywhn_label = label_recalc(label_list, label_path, label_name) # or xyxy
    
    # save coords_label for res_table
    coord_list_xyxy_label.append(coords_xyxy_label)
           
    # metric
    iou_list.append(bops.box_iou(pred_tensor, test_tensor).numpy())  
  
# finish time
end_time = time.time()

# start time minus final time
elapsed_time = end_time - start_time
print('Elapsed time: ', elapsed_time)

Starting...

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0023-aughor-inv.jpg: 512x640 1 class_0, 62.9ms
Speed: 0.0ms preprocess, 62.9ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0023-aughor.jpg: 512x640 1 class_0, 15.4ms
Speed: 1.0ms preprocess, 15.4ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0023-inv.jpg: 512x640 1 class_0, 26.2ms
Speed: 1.0ms preprocess, 26.2ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0023.jpg: 512x640 1 class_0, 15.9ms
Speed: 1.0ms preprocess, 15.9ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0035-aughor-inv.jpg: 512x640 1 cla

Speed: 0.0ms preprocess, 17.3ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0072-inv.jpg: 512x640 1 class_0, 15.1ms
Speed: 16.3ms preprocess, 15.1ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0072.jpg: 512x640 1 class_0, 15.8ms
Speed: 0.0ms preprocess, 15.8ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0077-aughor-inv.jpg: 512x640 1 class_0, 33.2ms
Speed: 0.0ms preprocess, 33.2ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0077-aughor.jpg: 512x640 1 class_0, 32.8ms
Speed: 0.0ms preprocess, 32.8ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Da

Speed: 0.0ms preprocess, 16.1ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1035-aughor-inv.jpg: 512x640 1 class_0, 3.9ms
Speed: 1.0ms preprocess, 3.9ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1035-aughor.jpg: 512x640 1 class_0, 13.3ms
Speed: 0.0ms preprocess, 13.3ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1035-inv.jpg: 512x640 1 class_0, 16.2ms
Speed: 0.0ms preprocess, 16.2ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1035.jpg: 512x640 1 class_0, 24.3ms
Speed: 2.0ms preprocess, 24.3ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Datas

Speed: 1.0ms preprocess, 5.2ms inference, 9.8ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1123-inv.jpg: 512x640 1 class_0, 17.5ms
Speed: 0.0ms preprocess, 17.5ms inference, 0.8ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1123.jpg: 512x640 1 class_0, 9.9ms
Speed: 0.0ms preprocess, 9.9ms inference, 12.6ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1124-aughor-inv.jpg: 512x640 1 class_0, 14.0ms
Speed: 0.0ms preprocess, 14.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1124-aughor.jpg: 512x640 1 class_0, 18.1ms
Speed: 2.0ms preprocess, 18.1ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Datas

Speed: 1.0ms preprocess, 12.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1224-aughor-inv.jpg: 512x640 1 class_0, 15.1ms
Speed: 1.4ms preprocess, 15.1ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1224-aughor.jpg: 512x640 1 class_0, 21.7ms
Speed: 13.6ms preprocess, 21.7ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1224-inv.jpg: 512x640 1 class_0, 18.7ms
Speed: 2.4ms preprocess, 18.7ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1224.jpg: 512x640 1 class_0, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 12.5ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dat

...


image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\1-helicopter\images\27042021_ 12_30@ds-0809.jpg: 512x640 1 class_0, 14.5ms
Speed: 1.0ms preprocess, 14.5ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\1-helicopter\images\27042021_ 12_30@ds-0900-aughor-inv.jpg: 512x640 1 class_0, 14.8ms
Speed: 1.7ms preprocess, 14.8ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\1-helicopter\images\27042021_ 12_30@ds-0900-aughor.jpg: 512x640 1 class_0, 14.7ms
Speed: 1.4ms preprocess, 14.7ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\1-helicopter\images\27042021_ 12_30@ds-0900-inv.jpg: 512x640 1 class_0, 15.8ms
Speed: 2.0ms preprocess, 15.8ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\1-helicopter\images\27042021_ 12_30@ds-0900.jpg: 5

Speed: 0.0ms preprocess, 16.5ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0009-aughor-inv.jpg: 512x640 1 class_0, 15.8ms
Speed: 0.0ms preprocess, 15.8ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0009-aughor.jpg: 512x640 1 class_0, 16.7ms
Speed: 0.0ms preprocess, 16.7ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0009-inv.jpg: 512x640 1 class_0, 16.6ms
Speed: 0.0ms preprocess, 16.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0009.jpg: 512x640 1 class_0, 17.2ms
Speed: 0.0ms preprocess, 17.2ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  

Speed: 0.6ms preprocess, 13.2ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0201.jpg: 512x640 1 class_0, 4.7ms
Speed: 1.0ms preprocess, 4.7ms inference, 10.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0252-aughor-inv.jpg: 512x640 1 class_0, 11.1ms
Speed: 0.8ms preprocess, 11.1ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0252-aughor.jpg: 512x640 (no detections), 15.0ms
Speed: 0.0ms preprocess, 15.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0252-inv.jpg: 512x640 1 class_0, 17.1ms
Speed: 0.0ms preprocess, 17.1ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\imag

Speed: 0.0ms preprocess, 16.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0320-inv.jpg: 512x640 1 class_0, 17.3ms
Speed: 2.0ms preprocess, 17.3ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0320.jpg: 512x640 1 class_0, 14.2ms
Speed: 1.0ms preprocess, 14.2ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0322-aughor-inv.jpg: 512x640 1 class_0, 15.9ms
Speed: 0.0ms preprocess, 15.9ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0322-aughor.jpg: 512x640 1 class_0, 14.6ms
Speed: 1.0ms preprocess, 14.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  

Speed: 0.0ms preprocess, 16.5ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0426-aughor.jpg: 512x640 1 class_0, 13.9ms
Speed: 0.7ms preprocess, 13.9ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0426-inv.jpg: 512x640 1 class_0, 5.3ms
Speed: 1.0ms preprocess, 5.3ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0426.jpg: 512x640 1 class_0, 29.1ms
Speed: 0.0ms preprocess, 29.1ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0443-aughor-inv.jpg: 512x640 1 class_0, 14.4ms
Speed: 1.0ms preprocess, 14.4ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 

In [95]:
print(f'It took {elapsed_time / 60} minutes for inference of {len(val_list)} images')

It took 1.7858418901761373 minutes for inference of 3712 images


In [96]:
res_tab = pd.DataFrame(columns = ['name', 'coord_list_xywhn_inf', 'coord_list_xyxy_inf', 'coord_list_xyxy_label', 'iou'])

res_tab['name'] = name_list
res_tab['coord_list_xywhn_inf'] = coord_list_xywhn_inference
res_tab['coord_list_xyxy_inf'] = coord_list_xyxy_inference
res_tab['coord_list_xyxy_label'] = coord_list_xyxy_label
res_tab['iou'] = iou_list

res_tab['iou_int'] = ''
for i in range(len(res_tab)):
    res_tab['iou_int'][i] = res_tab['iou'][i][0][0]
    
res_tab

,name,coord_list_xywhn_inf,coord_list_xyxy_inf,coord_list_xyxy_label,iou,iou_int
0,14052021_(10)@ds-0023-aughor-inv.jpg,"[0.370280385017395, 0.56596839427948, 0.034926...","[225.80308532714844, 284.43212890625, 248.1558...","[224.99840000000003, 282.99776, 248.0000000000...",[[0.8365339]],0.836534
1,14052021_(10)@ds-0023-aughor.jpg,"[0.37001943588256836, 0.5656412839889526, 0.03...","[226.0421142578125, 284.47705078125, 247.58276...","[224.99840000000003, 282.99776, 248.0000000000...",[[0.80080736]],0.800807
2,14052021_(10)@ds-0023-inv.jpg,"[0.6302878260612488, 0.5659416913986206, 0.032...","[393.0067443847656, 284.3708801269531, 413.761...","[391.99999999999994, 282.99776, 415.0015999999...",[[0.7899238]],0.789924
3,14052021_(10)@ds-0023.jpg,"[0.6298258900642395, 0.5657966136932373, 0.032...","[392.7915344238281, 284.2500305175781, 413.385...","[391.99999999999994, 282.99776, 415.0015999999...",[[0.7944021]],0.794402
4,14052021_(10)@ds-0035-aughor-inv.jpg,"[0.3680151402950287, 0.5661839246749878, 0.033...","[224.90614318847656, 284.63470458984375, 246.1...","[224.00000000000003, 284.00128, 247.0016000000...",[[0.8605073]],0.860507
...,...,...,...,...,...,...
3707,Φ¿α«¬«Ñ »«½Ñ 1 ¬¼@ds-0506.jpg,"[0.38540950417518616, 0.46047478914260864, 0.0...","[241.0895538330078, 232.91883850097656, 252.23...","[240.99840000000003, 233.00096000000002, 252.0...",[[0.8547017]],0.854702
3708,Φ¿α«¬«Ñ »«½Ñ 1 ¬¼@ds-0510-aughor-inv.jpg,"[0.6015133261680603, 0.459275484085083, 0.0175...","[379.3407897949219, 232.00933837890625, 390.59...","[380.0, 232.00256000000002, 391.0016, 238.0032...",[[0.86787283]],0.867873
3709,Φ¿α«¬«Ñ »«½Ñ 1 ¬¼@ds-0510-aughor.jpg,"[0.6014377474784851, 0.45976221561431885, 0.01...","[379.1883239746094, 232.48675537109375, 390.65...","[380.0, 232.00256000000002, 391.0016, 238.0032...",[[0.7939794]],0.793979
3710,Φ¿α«¬«Ñ »«½Ñ 1 ¬¼@ds-0510-inv.jpg,"[0.398375928401947, 0.45953160524368286, 0.017...","[249.24996948242188, 232.10971069335938, 260.6...","[248.99839999999998, 232.00256000000002, 260.0...",[[0.84380686]],0.843807


In [97]:
np.mean(res_tab['iou_int'])

0.8225373892948545

In [98]:
# check model on new videos and images

# plane
plane_video_name = ['14052021_(t16)@ds-', '14052021_(t15)@ds-', 'æá¼«½ÑΓ êè ¬á¡á½ 1@ds-']

plane_image_check_list = []

plane_image_count_df = pd.read_csv(os.path.join(project_path, 'plane_image_count_df.csv'))

for i in range(len(plane_video_name)):
    tab = plane_image_count_df[plane_image_count_df['video_name'] == plane_video_name[i]]
    plane_image_check_list.extend(tab['plane_list'])
    
plane_image_check_list

['14052021_(t16)@ds-0048-aughor-inv.jpg',
 '14052021_(t16)@ds-0034-inv.jpg',
 '14052021_(t16)@ds-0007-aughor-inv.jpg',
 '14052021_(t16)@ds-0101.jpg',
 '14052021_(t16)@ds-0022-inv.jpg',
 '14052021_(t16)@ds-0107-aughor.jpg',
 '14052021_(t16)@ds-0158-aughor-inv.jpg',
 '14052021_(t16)@ds-0137-aughor.jpg',
 '14052021_(t16)@ds-0064.jpg',
 '14052021_(t16)@ds-0192-aughor-inv.jpg',
 '14052021_(t16)@ds-0076.jpg',
 '14052021_(t16)@ds-0022-aughor.jpg',
 '14052021_(t16)@ds-0066-inv.jpg',
 '14052021_(t16)@ds-0202-aughor.jpg',
 '14052021_(t16)@ds-0099.jpg',
 '14052021_(t16)@ds-0067-aughor.jpg',
 '14052021_(t16)@ds-0098.jpg',
 '14052021_(t16)@ds-0136-aughor.jpg',
 '14052021_(t16)@ds-0092-inv.jpg',
 '14052021_(t16)@ds-0190-inv.jpg',
 '14052021_(t16)@ds-0079-aughor-inv.jpg',
 '14052021_(t16)@ds-0115-aughor-inv.jpg',
 '14052021_(t16)@ds-0181.jpg',
 '14052021_(t16)@ds-0195-aughor-inv.jpg',
 '14052021_(t16)@ds-0097.jpg',
 '14052021_(t16)@ds-0038-aughor.jpg',
 '14052021_(t16)@ds-0020.jpg',
 '14052021_(t16)@

In [108]:
img_path = plane_img_path
label_list = plane_image_check_list

# start time
start_time = time.time()
print('Starting...')

name_list = []
coord_list_xywhn_inference = []
coord_list_xyxy_inference = []
coord_list_xyxy_label = []
iou_list = []

conf = 0.3
max_det = 1

for i in range(len(val_list)):
    name_base = val_list[i][:-4:]
    image_name = val_list[i]
    label_name = name_base + '.txt'
    
    # save name for res_table
    name_list.append(val_list[i])
    
    if val_list[i] in plane_img_listdir:
        img_path = plane_img_path
        label_path = plane_label_path
        label_list = plane_label_listdir        
    else:
        img_path = heli_img_path
        label_path = heli_label_path
        label_list = heli_label_listdir        

    # get inference coords
    pred_tensor, coords_xyxy, coords_xywhn = yolo_fast_inference(img_path, image_name, conf, max_det) # or xyxy
    
    # save coords for res_table
    coord_list_xywhn_inference.append(coords_xywhn)
    coord_list_xyxy_inference.append(coords_xyxy)
    
    # get label
    test_tensor, coords_xyxy_label, coords_xywhn_label = label_recalc(label_list, label_path, label_name) # or xyxy
    
    # save coords_label for res_table
    coord_list_xyxy_label.append(coords_xyxy_label)
           
    # metric
    iou_list.append(bops.box_iou(pred_tensor, test_tensor).numpy())  
  
# finish time
end_time = time.time()

# start time minus final time
elapsed_time = end_time - start_time
print('Elapsed time: ', elapsed_time)

Starting...

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0023-aughor-inv.jpg: 512x640 1 class_0, 80.9ms
Speed: 4.5ms preprocess, 80.9ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0023-aughor.jpg: 512x640 1 class_0, 76.4ms
Speed: 1.2ms preprocess, 76.4ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0023-inv.jpg: 512x640 1 class_0, 17.6ms
Speed: 2.0ms preprocess, 17.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0023.jpg: 512x640 1 class_0, 16.6ms
Speed: 1.0ms preprocess, 16.6ms inference, 3.8ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0035-aughor-inv.jpg: 512x640 1 cla

Speed: 1.0ms preprocess, 37.1ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0072-inv.jpg: 512x640 1 class_0, 22.1ms
Speed: 0.0ms preprocess, 22.1ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0072.jpg: 512x640 1 class_0, 16.6ms
Speed: 1.0ms preprocess, 16.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0077-aughor-inv.jpg: 512x640 1 class_0, 7.5ms
Speed: 1.0ms preprocess, 7.5ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0077-aughor.jpg: 512x640 1 class_0, 16.8ms
Speed: 0.0ms preprocess, 16.8ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Datas

Speed: 11.7ms preprocess, 8.1ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1035-aughor-inv.jpg: 512x640 1 class_0, 14.0ms
Speed: 0.0ms preprocess, 14.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1035-aughor.jpg: 512x640 1 class_0, 24.7ms
Speed: 2.0ms preprocess, 24.7ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1035-inv.jpg: 512x640 1 class_0, 16.3ms
Speed: 0.0ms preprocess, 16.3ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1035.jpg: 512x640 1 class_0, 33.1ms
Speed: 0.0ms preprocess, 33.1ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dat

Speed: 2.0ms preprocess, 17.5ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1123-inv.jpg: 512x640 1 class_0, 27.3ms
Speed: 11.5ms preprocess, 27.3ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1123.jpg: 512x640 1 class_0, 14.0ms
Speed: 0.8ms preprocess, 14.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1124-aughor-inv.jpg: 512x640 1 class_0, 12.5ms
Speed: 0.0ms preprocess, 12.5ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1124-aughor.jpg: 512x640 1 class_0, 14.7ms
Speed: 1.0ms preprocess, 14.7ms inference, 1.3ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Da

Speed: 1.0ms preprocess, 20.2ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1224-aughor-inv.jpg: 512x640 1 class_0, 25.5ms
Speed: 2.0ms preprocess, 25.5ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1224-aughor.jpg: 512x640 1 class_0, 13.1ms
Speed: 1.0ms preprocess, 13.1ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1224-inv.jpg: 512x640 1 class_0, 4.3ms
Speed: 11.9ms preprocess, 4.3ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1224.jpg: 512x640 1 class_0, 15.7ms
Speed: 0.0ms preprocess, 15.7ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Data

...


image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\1-helicopter\images\27042021_ 12_30@ds-0809.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\1-helicopter\images\27042021_ 12_30@ds-0900-aughor-inv.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 15.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\1-helicopter\images\27042021_ 12_30@ds-0900-aughor.jpg: 512x640 1 class_0, 15.6ms
Speed: 15.6ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\1-helicopter\images\27042021_ 12_30@ds-0900-inv.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\1-helicopter\images\27042021_ 12_30@ds-0900.jpg:

Speed: 0.0ms preprocess, 31.2ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0009-aughor-inv.jpg: 512x640 1 class_0, 46.8ms
Speed: 0.0ms preprocess, 46.8ms inference, 15.6ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0009-aughor.jpg: 512x640 1 class_0, 31.2ms
Speed: 0.0ms preprocess, 31.2ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0009-inv.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0009.jpg: 512x640 1 class_0, 0.0ms
Speed: 9.5ms preprocess, 0.0ms inference, 15.6ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  

Speed: 0.0ms preprocess, 6.5ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0201.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0252-aughor-inv.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0252-aughor.jpg: 512x640 (no detections), 31.1ms
Speed: 0.0ms preprocess, 31.1ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0252-inv.jpg: 512x640 1 class_0, 15.6ms
Speed: 15.6ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\ima

Speed: 0.0ms preprocess, 16.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0320-inv.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0320.jpg: 512x640 1 class_0, 31.2ms
Speed: 0.0ms preprocess, 31.2ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0322-aughor-inv.jpg: 512x640 1 class_0, 16.0ms
Speed: 15.6ms preprocess, 16.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0322-aughor.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\ 

Speed: 0.0ms preprocess, 4.5ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0426-aughor.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0426-inv.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0426.jpg: 512x640 1 class_0, 47.3ms
Speed: 0.0ms preprocess, 47.3ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0443-aughor-inv.jpg: 512x640 1 class_0, 31.2ms
Speed: 0.0ms preprocess, 31.2ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1

In [100]:
# helicopters
heli_video = '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-'

heli_image_count_df = pd.read_csv(os.path.join(project_path, 'heli_image_count_df.csv'))

tab = heli_image_count_df[heli_image_count_df['video_name'] == heli_video]
heli_image_check_list = list(tab['heli_list'])
    
heli_image_check_list

['27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0002-aughor-inv.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0002-aughor.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0002-inv.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0002.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0003-aughor-inv.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0003-aughor.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0003-inv.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0003.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0004-aughor-inv.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0004-aughor.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0004-inv.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0004.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0005-aughor-inv.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0005-aughor.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0005-inv.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0005.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0006-aughor-inv.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0006-aughor.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0006-inv.jpg',
 '27042021_óÑαΓ«½ÑΓ¡¿¿» 12_35@ds-0006.jpg',
 '27042021

In [109]:
img_path = heli_img_path
label_list = heli_image_check_list

# start time
start_time = time.time()
print('Starting...')

conf = 0.3
max_det = 1

for i in range(len(val_list)):
    name_base = val_list[i][:-4:]
    image_name = val_list[i]
    label_name = name_base + '.txt'
    
    # save name for res_table
    name_list.append(val_list[i])
    
    if val_list[i] in plane_img_listdir:
        img_path = plane_img_path
        label_path = plane_label_path
        label_list = plane_label_listdir        
    else:
        img_path = heli_img_path
        label_path = heli_label_path
        label_list = heli_label_listdir        

    # get inference coords
    pred_tensor, coords_xyxy, coords_xywhn = yolo_fast_inference(img_path, image_name, conf, max_det) # or xyxy
    
    # save coords for res_table
    coord_list_xywhn_inference.append(coords_xywhn)
    coord_list_xyxy_inference.append(coords_xyxy)
    
    # get label
    test_tensor, coords_xyxy_label, coords_xywhn_label = label_recalc(label_list, label_path, label_name) # or xyxy
    
    # save coords_label for res_table
    coord_list_xyxy_label.append(coords_xyxy_label)
           
    # metric
    iou_list.append(bops.box_iou(pred_tensor, test_tensor).numpy())  
  
# finish time
end_time = time.time()

# start time minus final time
elapsed_time = end_time - start_time
print('Elapsed time: ', elapsed_time)

Starting...

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0023-aughor-inv.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0023-aughor.jpg: 512x640 1 class_0, 16.0ms
Speed: 0.0ms preprocess, 16.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0023-inv.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0023.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0035-aughor-inv.jpg: 512x640 1 cla

Speed: 0.0ms preprocess, 30.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0072-inv.jpg: 512x640 1 class_0, 8.8ms
Speed: 0.0ms preprocess, 8.8ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0072.jpg: 512x640 1 class_0, 29.8ms
Speed: 0.0ms preprocess, 29.8ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0077-aughor-inv.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-0077-aughor.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Datas

Speed: 0.0ms preprocess, 25.5ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1035-aughor-inv.jpg: 512x640 1 class_0, 7.9ms
Speed: 2.0ms preprocess, 7.9ms inference, 15.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1035-aughor.jpg: 512x640 1 class_0, 29.1ms
Speed: 2.0ms preprocess, 29.1ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1035-inv.jpg: 512x640 1 class_0, 8.8ms
Speed: 1.0ms preprocess, 8.8ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1035.jpg: 512x640 1 class_0, 15.0ms
Speed: 1.0ms preprocess, 15.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Datase

Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1123-inv.jpg: 512x640 1 class_0, 14.5ms
Speed: 1.5ms preprocess, 14.5ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1123.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1124-aughor-inv.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1124-aughor.jpg: 512x640 1 class_0, 9.1ms
Speed: 0.0ms preprocess, 9.1ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Datas

Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1224-aughor-inv.jpg: 512x640 1 class_0, 16.6ms
Speed: 0.0ms preprocess, 16.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1224-aughor.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1224-inv.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\14052021_(10)@ds-1224.jpg: 512x640 1 class_0, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dat

...


image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\1-helicopter\images\27042021_ 12_30@ds-0809.jpg: 512x640 1 class_0, 21.5ms
Speed: 0.0ms preprocess, 21.5ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\1-helicopter\images\27042021_ 12_30@ds-0900-aughor-inv.jpg: 512x640 1 class_0, 18.7ms
Speed: 0.0ms preprocess, 18.7ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\1-helicopter\images\27042021_ 12_30@ds-0900-aughor.jpg: 512x640 1 class_0, 14.5ms
Speed: 0.0ms preprocess, 14.5ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\1-helicopter\images\27042021_ 12_30@ds-0900-inv.jpg: 512x640 1 class_0, 5.2ms
Speed: 2.0ms preprocess, 5.2ms inference, 8.6ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\1-helicopter\images\27042021_ 12_30@ds-0900.jpg: 512

Speed: 0.0ms preprocess, 26.2ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0009-aughor-inv.jpg: 512x640 1 class_0, 8.2ms
Speed: 1.8ms preprocess, 8.2ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0009-aughor.jpg: 512x640 1 class_0, 16.1ms
Speed: 1.0ms preprocess, 16.1ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0009-inv.jpg: 512x640 1 class_0, 15.3ms
Speed: 0.0ms preprocess, 15.3ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0009.jpg: 512x640 1 class_0, 4.3ms
Speed: 6.1ms preprocess, 4.3ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @d

Speed: 0.7ms preprocess, 11.4ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0201.jpg: 512x640 1 class_0, 17.3ms
Speed: 2.0ms preprocess, 17.3ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0252-aughor-inv.jpg: 512x640 1 class_0, 14.2ms
Speed: 0.9ms preprocess, 14.2ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0252-aughor.jpg: 512x640 (no detections), 6.0ms
Speed: 13.4ms preprocess, 6.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0252-inv.jpg: 512x640 1 class_0, 13.7ms
Speed: 1.7ms preprocess, 13.7ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\imag

Speed: 0.5ms preprocess, 14.6ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0320-inv.jpg: 512x640 1 class_0, 17.0ms
Speed: 9.5ms preprocess, 17.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0320.jpg: 512x640 1 class_0, 11.1ms
Speed: 0.0ms preprocess, 11.1ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0322-aughor-inv.jpg: 512x640 1 class_0, 13.9ms
Speed: 1.0ms preprocess, 13.9ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0322-aughor.jpg: 512x640 1 class_0, 13.4ms
Speed: 0.0ms preprocess, 13.4ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  

Speed: 10.2ms preprocess, 13.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0426-aughor.jpg: 512x640 1 class_0, 14.0ms
Speed: 1.0ms preprocess, 14.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0426-inv.jpg: 512x640 1 class_0, 5.0ms
Speed: 7.2ms preprocess, 5.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0426.jpg: 512x640 1 class_0, 14.9ms
Speed: 0.0ms preprocess, 14.9ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 @ds-0443-aughor-inv.jpg: 512x640 1 class_0, 4.1ms
Speed: 1.0ms preprocess, 4.1ms inference, 10.9ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 F:\DS\_10.2024\Dataset\Dataset_photo\0-plane\images\  1 

In [110]:
res_tab_new_video = pd.DataFrame(columns = ['name', 'coord_list_xywhn_inf', 'coord_list_xyxy_inf', 'coord_list_xyxy_label', 'iou'])

res_tab_new_video['name'] = name_list
res_tab_new_video['coord_list_xywhn_inf'] = coord_list_xywhn_inference
res_tab_new_video['coord_list_xyxy_inf'] = coord_list_xyxy_inference
res_tab_new_video['coord_list_xyxy_label'] = coord_list_xyxy_label
res_tab_new_video['iou'] = iou_list

res_tab_new_video['iou_int'] = ''
for i in range(len(res_tab_new_video)):
    res_tab_new_video['iou_int'][i] = res_tab_new_video['iou'][i][0][0]
    
res_tab_new_video['label'] = 0
res_tab_new_video

,name,coord_list_xywhn_inf,coord_list_xyxy_inf,coord_list_xyxy_label,iou,iou_int,label
0,14052021_(10)@ds-0023-aughor-inv.jpg,"[0.370280385017395, 0.56596839427948, 0.034926...","[225.80308532714844, 284.43212890625, 248.1558...","[224.99840000000003, 282.99776, 248.0000000000...",[[0.8365339]],0.836534,0
1,14052021_(10)@ds-0023-aughor.jpg,"[0.37001943588256836, 0.5656412839889526, 0.03...","[226.0421142578125, 284.47705078125, 247.58276...","[224.99840000000003, 282.99776, 248.0000000000...",[[0.80080736]],0.800807,0
2,14052021_(10)@ds-0023-inv.jpg,"[0.6302878260612488, 0.5659416913986206, 0.032...","[393.0067443847656, 284.3708801269531, 413.761...","[391.99999999999994, 282.99776, 415.0015999999...",[[0.7899238]],0.789924,0
3,14052021_(10)@ds-0023.jpg,"[0.6298258900642395, 0.5657966136932373, 0.032...","[392.7915344238281, 284.2500305175781, 413.385...","[391.99999999999994, 282.99776, 415.0015999999...",[[0.7944021]],0.794402,0
4,14052021_(10)@ds-0035-aughor-inv.jpg,"[0.3680151402950287, 0.5661839246749878, 0.033...","[224.90614318847656, 284.63470458984375, 246.1...","[224.00000000000003, 284.00128, 247.0016000000...",[[0.8605073]],0.860507,0
...,...,...,...,...,...,...,...
7419,Φ¿α«¬«Ñ »«½Ñ 1 ¬¼@ds-0506.jpg,"[0.38540950417518616, 0.46047478914260864, 0.0...","[241.0895538330078, 232.91883850097656, 252.23...","[240.99840000000003, 233.00096000000002, 252.0...",[[0.8547017]],0.854702,0
7420,Φ¿α«¬«Ñ »«½Ñ 1 ¬¼@ds-0510-aughor-inv.jpg,"[0.6015133261680603, 0.459275484085083, 0.0175...","[379.3407897949219, 232.00933837890625, 390.59...","[380.0, 232.00256000000002, 391.0016, 238.0032...",[[0.86787283]],0.867873,0
7421,Φ¿α«¬«Ñ »«½Ñ 1 ¬¼@ds-0510-aughor.jpg,"[0.6014377474784851, 0.45976221561431885, 0.01...","[379.1883239746094, 232.48675537109375, 390.65...","[380.0, 232.00256000000002, 391.0016, 238.0032...",[[0.7939794]],0.793979,0
7422,Φ¿α«¬«Ñ »«½Ñ 1 ¬¼@ds-0510-inv.jpg,"[0.398375928401947, 0.45953160524368286, 0.017...","[249.24996948242188, 232.10971069335938, 260.6...","[248.99839999999998, 232.00256000000002, 260.0...",[[0.84380686]],0.843807,0


In [111]:
np.mean(res_tab_new_video['iou_int'])

0.8225376523774246

##### Test data

In [ ]:
# test data paths
test_path = os.path.join(project_path, 'test_data')
test_path_list = os.listdir(test_path)

# test data inference

# start time
start_time = time.time()
print('Starting...')

conf = 0.3
max_det = 1

for i in range(len(test_path_list)):
    image_name = val_list[i]
    
    # save name for res_table
    name_list.append(image_name)    

    # get inference coords
    pred_tensor, coords_xyxy, coords_xywhn = yolo_fast_inference(test_path, image_name, conf, max_det) # or xyxy
    
    # save coords for res_table
    coord_list_xywhn_inference.append(coords_xywhn)
    coord_list_xyxy_inference.append(coords_xyxy)
    
# finish time
end_time = time.time()

# start time minus final time
elapsed_time = end_time - start_time
print('Elapsed time: ', elapsed_time)

In [ ]:
# final_res_table
final_res_table = pd.DataFrame(columns = ['name', 'coord_list_xywhn_inf', 'coord_list_xyxy_inf', 'label'])

final_res_table['name'] = name_list
final_res_table['coord_list_xywhn_inf'] = coord_list_xywhn_inference
final_res_table['coord_list_xyxy_inf'] = coord_list_xyxy_inference   
final_res_table

In [ ]:
# save final_res_table
final_res_table.to_csv(os.path.join(project_path, 'final_res_table.csv'), index = False)

##### Comments
'coord_list_xywhn_inf' - for submission

'coord_list_xyxy_inf' - for image_crop for classifier


##### Classification

In [ ]:
# load res_table
final_res_table = pd.read_csv(os.path.join(project_path, 'final_res_table.csv'))
final_res_table

In [ ]:
# Перебор и обрезка изображений for large model
image_np = np.array(final_res_table['name'])
bbox_inf_np = np.array(final_res_table['coord_list_xyxy_inf'])

img_path = ..... # path to your folder with test images

for i in range(len(image_np)):
    # open image
    image = Image.open(os.path.join(img_path, image_np[i]))
    
    # define coordinates for crop (if model didn't find the object, for now we can use label coordinates to train)
    if bbox_inf_np[i] != [0, 0, 0, 0]:
        x1 = bbox_inf_np[i][0]
        y1 = bbox_inf_np[i][1]
        x2 = bbox_inf_np[i][2]
        y2 = bbox_inf_np[i][3]
        
        # crop image    
        cropped_image = image.crop((x1, y1, x2, y2))
    
        # classify
        ....
        res = ....
        
        final_res_table['label'][i] = res
    else:
        final_res_table['label'][i] = 0

#### Submission

In [53]:
import json
from json import loads, dumps

In [55]:
'''
fp = open(os.path.join(project_path, "test_submit.json"), "r")

# Decoded result
result = json.load(fp)
result
'''

[{'filename': '<filename>.jpg',
  'objects': [{'obj_class': '0',
    'x': '0',
    'y': '0',
    'width': '1',
    'height': '1'}]},
 {'filename': '<filename2>.jpg', 'objects': [{}]},
 {'filename': '<filename3>.jpg',
  'objects': [{'obj_class': '0',
    'x': '0',
    'y': '0',
    'width': '1',
    'height': '1'},
   {'obj_class': '1', 'x': '10', 'y': '10', 'width': '10', 'height': '10'},
   {'obj_class': '0', 'x': '20', 'y': '20', 'width': '20', 'height': '20'}]}]

In [66]:
submission = []

for i in range(len(final_res_table)):     
    submission.append({'filename': final_res_table['name'][i],
                       'objects': [{'obj_class': str(final_res_table['label'][i]),
                          'x': str(final_res_table['coord_list_xywhn_inf'][i][0]),
                          'y': str(final_res_table['coord_list_xywhn_inf'][i][1]),
                          'width': str(final_res_table['coord_list_xywhn_inf'][i][2]),
                          'height': str(final_res_table['coord_list_xywhn_inf'][i][3])}]})
    
with open(os.path.join(project_path, "submission_v1.json"), "w") as outfile: 
    json.dump(submission, outfile)

In [112]:
# trial

submission = []

for i in range(len(res_tab_new_video)):
      
    submission.append({'filename': res_tab_new_video['name'][i],
                  'objects': [{'obj_class': str(res_tab_new_video['label'][i]),
                     'x': str(res_tab_new_video['coord_list_xywhn_inf'][i][0]),
                     'y': str(res_tab_new_video['coord_list_xywhn_inf'][i][1]),
                     'width': str(res_tab_new_video['coord_list_xywhn_inf'][i][2]),
                     'height': str(res_tab_new_video['coord_list_xywhn_inf'][i][3])}]})
    
with open(os.path.join(project_path, "submission_v0.json"), "w") as outfile: 
    json.dump(submission, outfile)

In [113]:
# trial check
fp = open(os.path.join(project_path, "submission_v0.json"), "r")

# Decoded result
result = json.load(fp)
result

[{'filename': '14052021_(10)@ds-0023-aughor-inv.jpg',
  'objects': [{'obj_class': '0',
    'x': '0.370280385017395',
    'y': '0.56596839427948',
    'width': '0.03492612764239311',
    'height': '0.020873725414276123'}]},
 {'filename': '14052021_(10)@ds-0023-aughor.jpg',
  'objects': [{'obj_class': '0',
    'x': '0.37001943588256836',
    'y': '0.5656412839889526',
    'width': '0.0336572639644146',
    'height': '0.020044028759002686'}]},
 {'filename': '14052021_(10)@ds-0023-inv.jpg',
  'objects': [{'obj_class': '0',
    'x': '0.6302878260612488',
    'y': '0.5659416913986206',
    'width': '0.03242964670062065',
    'height': '0.02105957269668579'}]},
 {'filename': '14052021_(10)@ds-0023.jpg',
  'objects': [{'obj_class': '0',
    'x': '0.6298258900642395',
    'y': '0.5657966136932373',
    'width': '0.03217821195721626',
    'height': '0.021241605281829834'}]},
 {'filename': '14052021_(10)@ds-0035-aughor-inv.jpg',
  'objects': [{'obj_class': '0',
    'x': '0.3680151402950287',
    

##### Additional

In [ ]:
# produce some videos

# Define the parameters
imgsz = 640
conf = 0.3
#iou = 0.7

# Load the YOLOv8 model
#model = YOLO("yolov8m.pt") 

fps_counter = FPS_Counter(calc_time_perion_N_frames=10) # Определяем размер усреднения

# Инициализация 
cap = cv2.VideoCapture(r"F:\DS\Хахатон_10.2024\Dataset\Dataset_video\14052021_(t16).mp4")

# Проверка, успешно ли открыто видео
if not cap.isOpened():
    print("Не удалось открыть видео")
    exit()

# Получение исходного FPS и размеров кадра
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Создание объекта VideoWriter для сохранения результата
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Кодек для MP4
out = cv2.VideoWriter(os.path.join(project_path, '14052021_(t16)_res_large.mp4'), fourcc, fps, (width, height))

while True:
    # Захват кадра 
    ret, frame = cap.read()

    # Проверка, успешно ли захвачен кадр
    if not ret:
        print("Не удалось получить кадр")
        break

    frame = visualize_results_usual_yolo_inference(
        frame,
        model,
        imgsz,
        conf,
        #iou,
        segment=False,
        delta_colors=13,
        thickness=2,
        font_scale=0.1,
        show_boxes=True,
        show_class=False,
        show_confidences=False,
        return_image_array=True
    )

    # Ресайз
    scale = 0.5
    frame_resized = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)
    frame_resized = calc_and_show_fps(frame_resized, fps_counter)

    # Отображение кадра
    #cv2.imshow('video', frame_resized)

    # Запись кадра в выходной видеофайл
    out.write(frame)

    # Выход из цикла по нажатию клавиши 'q'
    #if cv2.waitKey(1) & 0xFF == ord('q'):
    #    break

# Освобождение ресурсов
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
# produce some videos

# Define the parameters
imgsz = 640
conf = 0.3
#iou = 0.7

# Load the YOLOv8 model
#model = YOLO("yolov8m.pt") 

fps_counter = FPS_Counter(calc_time_perion_N_frames=10) # Определяем размер усреднения

# Инициализация 
cap = cv2.VideoCapture(r"F:\DS\Хахатон_10.2024\Dataset\Dataset_video\27042021_óÑαΓ«½ÑΓ¡¿¿» 12_30.mp4")

# Проверка, успешно ли открыто видео
if not cap.isOpened():
    print("Не удалось открыть видео")
    exit()

# Получение исходного FPS и размеров кадра
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Создание объекта VideoWriter для сохранения результата
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Кодек для MP4
out = cv2.VideoWriter(os.path.join(project_path, '27042021_12_30_res_large.mp4'), fourcc, fps, (width, height))

while True:
    # Захват кадра 
    ret, frame = cap.read()

    # Проверка, успешно ли захвачен кадр
    if not ret:
        print("Не удалось получить кадр")
        break

    frame = visualize_results_usual_yolo_inference(
        frame,
        model,
        imgsz,
        conf,
        #iou,
        segment=False,
        delta_colors=13,
        thickness=2,
        font_scale=0.1,
        show_boxes=True,
        show_class=False,
        show_confidences=False,
        return_image_array=True
    )

    # Ресайз
    scale = 0.5
    frame_resized = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)
    frame_resized = calc_and_show_fps(frame_resized, fps_counter)

    # Отображение кадра
    #cv2.imshow('video', frame_resized)

    # Запись кадра в выходной видеофайл
    out.write(frame)

    # Выход из цикла по нажатию клавиши 'q'
    #if cv2.waitKey(1) & 0xFF == ord('q'):
    #    break

# Освобождение ресурсов
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
# produce some videos

# Define the parameters
imgsz = 640
conf = 0.3
#iou = 0.7

# Load the YOLOv8 model
#model = YOLO("yolov8m.pt") 

fps_counter = FPS_Counter(calc_time_perion_N_frames=10) # Определяем размер усреднения

# Инициализация 
cap = cv2.VideoCapture(r"F:\DS\Хахатон_10.2024\Dataset\Dataset_video\14052021_(10).mp4")

# Проверка, успешно ли открыто видео
if not cap.isOpened():
    print("Не удалось открыть видео")
    exit()

# Получение исходного FPS и размеров кадра
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Создание объекта VideoWriter для сохранения результата
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Кодек для MP4
out = cv2.VideoWriter(os.path.join(project_path, '14052021_(10)_res_large.mp4'), fourcc, fps, (width, height))

while True:
    # Захват кадра 
    ret, frame = cap.read()

    # Проверка, успешно ли захвачен кадр
    if not ret:
        print("Не удалось получить кадр")
        break

    frame = visualize_results_usual_yolo_inference(
        frame,
        model,
        imgsz,
        conf,
        #iou,
        segment=False,
        delta_colors=13,
        thickness=2,
        font_scale=0.1,
        show_boxes=True,
        show_class=False,
        show_confidences=False,
        return_image_array=True
    )

    # Ресайз
    scale = 0.5
    frame_resized = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)
    frame_resized = calc_and_show_fps(frame_resized, fps_counter)

    # Отображение кадра
    #cv2.imshow('video', frame_resized)

    # Запись кадра в выходной видеофайл
    out.write(frame)

    # Выход из цикла по нажатию клавиши 'q'
    #if cv2.waitKey(1) & 0xFF == ord('q'):
    #    break

# Освобождение ресурсов
cap.release()
out.release()
cv2.destroyAllWindows()